In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import Tracker
import cisco_fiscal_calendar as fc

In [2]:
def squeeze_customer_name(customer_name):
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [3]:
### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName

def make_fake_guid(guid, cavid, customername):
    """Returns either a GUID, a CAV ID
    """
    if pd.isnull(guid):
        if pd.isnull(cavid):
            return str(customername[:10])
        elif isinstance(cavid, float):
            return str(int(cavid))
        elif ',' in cavid:
            quicklist = list(cavid.split(','))
            return str(int(quicklist[0]))
        else:
            return str(int(cavid))
    elif type(guid) == float:
        return str(int(guid))
    
    elif isinstance(guid, str):
        return str(customername[:10])
    
    elif ',' in guid:
        quicklist = list(guid.split(','))
        return str(int(quicklist[0]))
    
    elif ';' in guid:
        quicklist = list(guid.split(';'))
        return str(int(quicklist[0]))
    
    elif '\r\n' in guid:
        quicklist = list(guid.split('\r\n'))
        return str(int(quicklist[0]))
    
    elif guid == '-':
        return str(customername[:10])
        
    elif str(guid).isalpha():
        return str(customername[:10])
    
    elif ' ' in guid:
        quicklist = list(guid.split(' '))
        if quicklist[0].isalpha():
            return str(customername[:10])
        else:
            return str(int(quicklist[0]))
    
    elif len(str(guid)) > 12:
        return str(guid)[:12]
    
    else:
        return str(guid)

In [4]:
finbi_path = r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.pkl'
fins = pd.read_pickle(finbi_path)

In [5]:
fins.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Partner', 'Booked Deal ID', 'CX Product Category',
       'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Week ID',
       'Fiscal Week of Year', 'ReportWeek', 'ProductType', 'TCV', 'ACV', 'Ind',
       'Request ID', 'Date Readout Done', 'Campaign Type', 'Compass Exists?',
       'Compass Claim?', 'Notes', 'Respondant_File', 'AddIncre', 'Campaign',
       'Target Campaign Flag', 'Partner Deal', 'Service Source',
       'Compass GUID Name', 'Compass Deal ID', 'Compass Advisor',
       'Readout Delivery Status', 'Items Recommended'],
      dtype='object')

In [6]:
fins['Fiscal Month'] = fins['Fiscal Week ID'].apply(lambda x: int(str(x)[:6]))

In [7]:
# fins['Fiscal Year'] = fins['Fiscal Week ID'].apply(lambda x: int(str(x)[:4]))

In [8]:
fins['GUID'] = fins['GUID'].astype('Int64')

In [9]:
fins['CAV ID'] = fins['CAV ID'].astype('Int64')

In [10]:
def concat_fin_names(cav_name, guid_name):
    if pd.isnull(cav_name):
        return guid_name
    else:
        return cav_name + '_' + guid_name

In [11]:
def get_account_name(cav_name, guid_name):
    customer_name = None
    if pd.isnull(cav_name) and pd.notnull(guid_name):
        customer_name = guid_name
    else:
        customer_name = cav_name
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [12]:
fins['FINBI Name'] = fins.apply(lambda x: concat_fin_names(x['CAV Name'], x['GUID Name']), axis=1)

In [13]:
fins['Squeezed'] = fins.apply(lambda x: get_account_name(x['CAV Name'],x['GUID Name']), axis=1)

In [14]:
# fins['Squeezed']

In [15]:
fins['Prepend'] = fins['Squeezed'].apply(lambda x: x[:2].upper())

In [16]:
fins['Prehash_Entity'] = fins.apply(lambda x: x['Prepend'] + str(x['GUID']), axis=1)

In [17]:
fins['Prehash_Entity']

0          CE180320
1          CE180320
2          CE180320
3          CE180320
4          CE180320
           ...     
2816        SI92372
2817     ST83880344
2818      TE2374518
2819        TH77043
2820    X5205204294
Name: Prehash_Entity, Length: 2821, dtype: object

In [18]:
# fins['FINBI Name'] = fins.loc[fins['CAV Name'].notnull()].apply(lambda x: x['GUID Name'] + '_' + x['CAV Name'], axis=1)

In [19]:
fins['Campaign'] = fins['Campaign'].str.upper()

In [20]:
fins.loc[fins['Sales Level 2']=='GLOBAL ENTERPRISE SEGMENT','Sales Level 2']='GES'

In [21]:
month_list = list(fins['Fiscal Month'].unique())

In [22]:
month_list.sort()

In [23]:
earliest_finbi_month = month_list[11] # we need 11 months plus a 30-day minimum booking window, so 12 months back
last_finbi_month = month_list[-1]

In [24]:
earliest_finbi_month, last_finbi_month

(202012, 202202)

In [25]:
A1 = month_list.index(earliest_finbi_month)
use_list = month_list[A1:]

In [26]:
use_list[-2]

202201

In [27]:
caldf = fc.calendar_df

In [28]:
removed_campaigns = ['REACTIVE - ACC DESIGNATION','REACTIVE - PARTNER','REACTIVE - GROWTH INITIATIVE',
                     'PROACTIVE - TAC LEAD','PROACTIVE - CS RESEARCH','PROACTIVE - PRESALES ALIGN PITSTOP EFFORT']

In [29]:
fins = fins.loc[~fins['Campaign'].isin(removed_campaigns)]

In [30]:
load_table = []  # table to load temp_table dataframes from the loop process

In [31]:
tracker = Tracker.main()

In [32]:
### Fix some tracker SL3 values to conform them to FINBI SL3 values
tracker.loc[tracker['Lvl3']=='GES-MWA','Lvl3']='GES MWA'
tracker.loc[tracker['Lvl3']=='MEXICO','Lvl3']='MEXICO-'
tracker.loc[tracker['Lvl3']=='SLED-SOUTHEAST OPERATION','Lvl3']='SLED-EAST AREA'
tracker.loc[tracker['Lvl3']=='SLED_WEST AREA','Lvl3']='SLED-WEST AREA'
tracker.loc[tracker['Lvl3']=='US COMM MISC L3','Lvl3']='US COMMERCIAL MISC'

In [33]:
tracker['Date Readout Done'].min()

Timestamp('2019-09-26 00:00:00')

In [34]:
for fiscal_period_id in use_list[:-1]:
    max_tracker_readout_date = pd.to_datetime(caldf.loc[caldf['finbi_fiscal_period_id'] == fiscal_period_id]['date'].max()) # The absolute MAX date for the finbi period
    min_tracker_readout_date = max_tracker_readout_date + dt.timedelta(days = -333)   # Looking 11 months back
    ten_months_ago = max_tracker_readout_date + dt.timedelta(days = -303)   # Looking 10 months back
    revised_max_tracker_readout_date = max_tracker_readout_date + dt.timedelta(days = -30)   #reset the MAX date for the period to be 30 days less, to account for the bookings minimum of 30 days out, for that booking window
    
    oldest_fiscal_week_id = caldf.loc[caldf['date']==ten_months_ago.isoformat()[:10]]['finbi_fiscal_week_id'].values[0]
    thirty_days_ago_week_id = caldf.loc[caldf['date']==revised_max_tracker_readout_date.isoformat()[:10]]['finbi_fiscal_week_id'].values[0]
    max_tracker_readout_date_week_id = caldf.loc[caldf['date']==max_tracker_readout_date.isoformat()[:10]]['finbi_fiscal_week_id'].values[0]
    
    fins['Report Month'] = fiscal_period_id
    fins['Sales Level 2'] = fins['Sales Level 2'].str.upper()
    fins['Sales Level 1'] = fins['Sales Level 1'].str.upper()
    
    
    fin_sl1           = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Sales Level 1'])[['Prehash_Entity']].nunique()
    fin_ct            = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Campaign Type'])[['Prehash_Entity']].nunique()
    fin_sl1_ct        = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Sales Level 1','Campaign Type'])[['Prehash_Entity']].nunique()
    fin_sl1_sl2_ct    = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Sales Level 1','Sales Level 2','Campaign Type'])[['Prehash_Entity']].nunique()
    fin_sl1_sl2       = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Sales Level 1','Sales Level 2'])[['Prehash_Entity']].nunique()
    fin_sl1_sl2_sl3   = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Sales Level 1','Sales Level 2','Sales Level 3'])[['Prehash_Entity']].nunique()
    fin_sl1_sl2_sl3_ct = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ].groupby(['Sales Level 1','Sales Level 2','Sales Level 3','Campaign Type'])[['Prehash_Entity']].nunique()
    overall_numerator = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= max_tracker_readout_date_week_id)) ][['Prehash_Entity']].nunique()[0]
    
    
    fin_sl1_ct.reset_index(inplace=True)
    fin_ct.reset_index(inplace=True)
    fin_sl1.reset_index(inplace=True)
    fin_sl1_sl2_ct.reset_index(inplace=True)
    fin_sl1_sl2.reset_index(inplace=True)
    fin_sl1_sl2_sl3.reset_index(inplace=True)
    fin_sl1_sl2_sl3_ct.reset_index(inplace=True)
    
    fin_sl1_ct.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    fin_ct.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    fin_sl1.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    fin_sl1_sl2_ct.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    fin_sl1_sl2.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    fin_sl1_sl2_sl3.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    fin_sl1_sl2_sl3_ct.rename(columns = {'Prehash_Entity':'FINBI Name'}, inplace=True)
    
    
    ### Set the SL and Campaign Dimension Level on the various dataframes
    fin_sl1_ct['Report Month'] = fiscal_period_id
    fin_sl1_ct['Dimension Grouping'] = 'SL1_CT'
    
    fin_ct['Report Month'] = fiscal_period_id
    fin_ct['Dimension Grouping'] = 'CT ONLY'
    
    fin_sl1['Report Month'] = fiscal_period_id
    fin_sl1['Dimension Grouping'] = 'SL1 ONLY'
    
    fin_sl1_sl2_ct['Report Month'] = fiscal_period_id
    fin_sl1_sl2_ct['Dimension Grouping'] = 'SL1_SL2_CT'
    
    fin_sl1_sl2['Report Month'] = fiscal_period_id
    fin_sl1_sl2['Dimension Grouping'] = 'SL1_SL2'
    
    fin_sl1_sl2_sl3['Report Month'] = fiscal_period_id
    fin_sl1_sl2_sl3['Dimension Grouping'] = 'SL1_SL2_SL3'

    fin_sl1_sl2_sl3_ct['Report Month'] = fiscal_period_id
    fin_sl1_sl2_sl3_ct['Dimension Grouping'] = 'SL1_SL2_SL3_CT'
    
    fin_sl1['Sales Level 1']        = fin_sl1['Sales Level 1'].str.upper()
    fin_ct['Campaign Type']         = fin_ct['Campaign Type'].str.upper()
    fin_sl1_ct['Sales Level 1']     = fin_sl1_ct['Sales Level 1'].str.upper()
    fin_sl1_sl2_ct['Sales Level 1'] = fin_sl1_sl2_ct['Sales Level 1'].str.upper()
    fin_sl1_sl2_ct['Sales Level 2'] = fin_sl1_sl2_ct['Sales Level 2'].str.upper()
    fin_sl1_sl2['Sales Level 1']    = fin_sl1_sl2['Sales Level 1'].str.upper()
    fin_sl1_sl2['Sales Level 2']    = fin_sl1_sl2['Sales Level 2'].str.upper()
    fin_sl1_sl2_sl3['Sales Level 1']    = fin_sl1_sl2_sl3['Sales Level 1'].str.upper()
    fin_sl1_sl2_sl3['Sales Level 2']    = fin_sl1_sl2_sl3['Sales Level 2'].str.upper()
    fin_sl1_sl2_sl3['Sales Level 3']    = fin_sl1_sl2_sl3['Sales Level 3'].str.upper()
    fin_sl1_sl2_sl3_ct['Sales Level 1']    = fin_sl1_sl2_sl3_ct['Sales Level 1'].str.upper()
    fin_sl1_sl2_sl3_ct['Sales Level 2']    = fin_sl1_sl2_sl3_ct['Sales Level 2'].str.upper()
    fin_sl1_sl2_sl3_ct['Sales Level 3']    = fin_sl1_sl2_sl3_ct['Sales Level 3'].str.upper()
    
    # Set Uniform Values for the Lvl Entries
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('UKI', case=False)), 'Lvl2 (Region)'] = 'EMEAR-UKI'
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('central', case=False)), 'Lvl2 (Region)'] = 'EMEAR-CENTRAL'
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('north', case=False)), 'Lvl2 (Region)'] = 'EMEAR-NORTH'
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('south', case=False)), 'Lvl2 (Region)'] = 'EMEAR-SOUTH'
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('ANZ', case=False)), 'Lvl2 (Region)'] = 'ANZ AREA'
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('japan', case=False)), 'Lvl2 (Region)'] = 'JAPAN__'
    tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('global enterprise segment', case=False)),'Lvl2 (Region)'] = 'GES'
    tracker.loc[(tracker['Lvl1'].notnull()) & (tracker['Lvl1'].str.contains('apjc__', case=False)),'Lvl1'] = 'APJC'
    
    trkr_condition1A = (tracker['Date Readout Done']>= min_tracker_readout_date)
    trkr_condition1B = (tracker['Date Readout Done']<= revised_max_tracker_readout_date)
    trkr_condition2 = ~(tracker['Compass Campaign Name'].isin(removed_campaigns))
    trkr_condition3 = ~(tracker['Readout Delivery Status'].isin(['Declined','Postponed']))
    trkr_condition4 = (tracker['Items Recommended'].notnull())
    #trkr_condition5 = (tracker['is_Targeting_Campaign']==True)
    
    compass_sl1         = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Lvl1'])[['Prehash_Entity']].nunique()
    compass_ct          = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Campaign Type'])[['Prehash_Entity']].nunique()
    compass_sl1_ct      = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Lvl1','Campaign Type'])[['Prehash_Entity']].nunique()
    compass_sl1_sl2_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Lvl1','Lvl2 (Region)','Campaign Type'])[['Prehash_Entity']].nunique()
    compass_sl1_sl2     = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Lvl1','Lvl2 (Region)'])[['Prehash_Entity']].nunique()
    compass_sl1_sl2_sl3     = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Lvl1','Lvl2 (Region)','Lvl3'])[['Prehash_Entity']].nunique()
    compass_sl1_sl2_sl3_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)].groupby(['Lvl1','Lvl2 (Region)','Lvl3','Campaign Type'])[['Prehash_Entity']].nunique()
    overall_denominator = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & (trkr_condition3 | trkr_condition4)][['Prehash_Entity']].nunique()[0]
    
    compass_sl1.reset_index(inplace=True)
    compass_ct.reset_index(inplace=True)
    compass_sl1_ct.reset_index(inplace=True)
    compass_sl1_sl2_ct.reset_index(inplace=True)
    compass_sl1_sl2.reset_index(inplace=True)
    compass_sl1_sl2_sl3.reset_index(inplace=True)
    compass_sl1_sl2_sl3_ct.reset_index(inplace=True)
    
    compass_sl1.rename(columns = {'Lvl1':'Sales Level 1','Prehash_Entity':'Compass Name'}, inplace=True)
    compass_ct.rename(columns = {'Prehash_Entity':'Compass Name'}, inplace=True)
    compass_sl1_ct.rename(columns = {'Lvl1':'Sales Level 1','Prehash_Entity':'Compass Name'}, inplace=True)
    compass_sl1_sl2_ct.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Prehash_Entity':'Compass Name'}, inplace=True)
    compass_sl1_sl2.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Prehash_Entity':'Compass Name'}, inplace=True)
    compass_sl1_sl2_sl3.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2', 'Lvl3':'Sales Level 3','Prehash_Entity':'Compass Name'}, inplace=True)
    compass_sl1_sl2_sl3_ct.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Lvl3':'Sales Level 3','Prehash_Entity':'Compass Name'}, inplace=True)
    
    combined_sl1 = pd.merge(fin_sl1, compass_sl1, on = ['Sales Level 1'])
    combined_ct = pd.merge(fin_ct, compass_ct, on=['Campaign Type'])
    combined_sl1_ct = pd.merge(fin_sl1_ct, compass_sl1_ct, on = ['Sales Level 1','Campaign Type'])
    combined_sl1_sl2_ct = pd.merge(fin_sl1_sl2_ct,compass_sl1_sl2_ct, on = ['Sales Level 1', 'Sales Level 2','Campaign Type'])
    combined_sl1_sl2 = pd.merge(fin_sl1_sl2,compass_sl1_sl2, on = ['Sales Level 1', 'Sales Level 2'])
    combined_sl1_sl2_sl3 = pd.merge(fin_sl1_sl2_sl3,compass_sl1_sl2_sl3, on = ['Sales Level 1', 'Sales Level 2','Sales Level 3'])
    combined_sl1_sl2_sl3_ct = pd.merge(fin_sl1_sl2_sl3_ct,compass_sl1_sl2_sl3_ct, on = ['Sales Level 1', 'Sales Level 2','Sales Level 3','Campaign Type'])
    
    combined_sl1_sl2_ct['Conversion Ratio %'] = (combined_sl1_sl2_ct['FINBI Name'] / combined_sl1_sl2_ct['Compass Name']) #* 100
    combined_sl1_ct['Conversion Ratio %'] = (combined_sl1_ct['FINBI Name'] / combined_sl1_ct['Compass Name']) #* 100
    combined_sl1['Conversion Ratio %'] = (combined_sl1['FINBI Name'] / combined_sl1['Compass Name']) #* 100
    combined_ct['Conversion Ratio %'] = (combined_ct['FINBI Name'] / combined_ct['Compass Name']) #* 100
    combined_sl1_sl2['Conversion Ratio %'] = (combined_sl1_sl2['FINBI Name'] / combined_sl1_sl2['Compass Name']) #* 100
    combined_sl1_sl2_sl3['Conversion Ratio %'] = (combined_sl1_sl2_sl3['FINBI Name'] / combined_sl1_sl2_sl3['Compass Name']) #* 100
    combined_sl1_sl2_sl3_ct['Conversion Ratio %'] = (combined_sl1_sl2_sl3_ct['FINBI Name'] / combined_sl1_sl2_sl3_ct['Compass Name']) #* 100
    
    combined_ct['Sales Level 1'] = pd.NA
    combined_ct['Sales Level 2'] = pd.NA
    combined_ct['Sales Level 3'] = pd.NA
    combined_ct = combined_ct[['Dimension Grouping', 'Report Month','Sales Level 1','Sales Level 2','Sales Level 3','Campaign Type', 'FINBI Name',  'Compass Name','Conversion Ratio %']]
    
    combined_sl1_ct['Sales Level 2'] = pd.NA
    combined_sl1_ct['Sales Level 3'] = pd.NA
    combined_sl1_ct = combined_sl1_ct[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]
    
    combined_sl1['Sales Level 2'] = pd.NA
    combined_sl1['Campaign Type'] = pd.NA
    combined_sl1['Sales Level 3'] = pd.NA
    combined_sl1 = combined_sl1[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]
    
    combined_sl1_sl2_ct['Sales Level 3'] = pd.NA
    combined_sl1_sl2_ct = combined_sl1_sl2_ct[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]
    
    combined_sl1_sl2['Campaign Type'] = pd.NA
    combined_sl1_sl2['Sales Level 3'] = pd.NA
    combined_sl1_sl2 = combined_sl1_sl2[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]
    
    combined_sl1_sl2_sl3['Campaign Type'] = pd.NA
    combined_sl1_sl2_sl3 = combined_sl1_sl2_sl3[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]
    
    overall_conversion_ratio = (overall_numerator / overall_denominator) #* 100
    overall_list = [['OVERALL',fiscal_period_id, pd.NA, pd.NA, pd.NA, pd.NA, overall_numerator, overall_denominator, overall_conversion_ratio]]
    overall_columns = ['Dimension Grouping', 'Report Month','Sales Level 1','Sales Level 2','Sales Level 3','Campaign Type','FINBI Name','Compass Name','Conversion Ratio %']
    overall = pd.DataFrame(data = overall_list, columns = overall_columns)
    
    temp_table = pd.concat([overall,combined_ct,combined_sl1, combined_sl1_ct, combined_sl1_sl2, combined_sl1_sl2_ct, combined_sl1_sl2_sl3, combined_sl1_sl2_sl3_ct])
    load_table.append(temp_table)
    
    #print(f'{fiscal_period_id}: {min_tracker_readout_date} as weekID {oldest_fiscal_week_id}, {max_tracker_readout_date}, {revised_max_tracker_readout_date} as weekID {thirty_days_ago_week_id}')
    

In [35]:
# tracker.loc[tracker['Lvl2 (Region)']=='GLOBAL ENTERPRISE SEGMENT']
tracker['Lvl2 (Region)'].unique()

array(['CANADA', 'AMERICAS_SP', 'LATIN AMERICA', 'GES',
       'US PS MARKET SEGMENT', 'US COMMERCIAL', 'EMEAR-CENTRAL',
       'EMEAR-SOUTH', 'EMEAR_GERMANY', 'JAPAN__', 'EMEAR_MEA',
       'EMEAR-UKI', 'ANZ AREA', 'EMEAR_SP', 'EMEAR-NORTH', 'INDIA_AREA',
       'APJ_SP', 'ASEAN_AREA', 'GREATER_CHINA', 'ROK_AREA',
       'AMERICAS_MISC', 'EMEAR_UNALLOCATED'], dtype=object)

In [36]:
fins['Sales Level 2'].unique()

array(['GES', 'LATIN AMERICA', 'US COMMERCIAL', 'CANADA',
       'US PS MARKET SEGMENT', 'EMEAR-SOUTH', 'EMEAR_MEA', 'EMEAR-UKI',
       'EMEAR-CENTRAL', 'EMEAR_GERMANY', 'GREATER_CHINA', 'INDIA_AREA',
       'ANZ AREA', 'JAPAN__', 'ASEAN_AREA', 'EMEAR-NORTH', 'APJ_SP',
       'EMEAR_SP', 'AMERICAS_SP', 'ROK_AREA'], dtype=object)

In [37]:
load_table = pd.concat(load_table)

In [38]:
load_table

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Sales Level 3,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,OVERALL,202012,<NA>,<NA>,<NA>,<NA>,90,670,0.134328
0,CT ONLY,202012,<NA>,<NA>,<NA>,FIELD,65,557,0.116697
1,CT ONLY,202012,<NA>,<NA>,<NA>,TARGETING,26,132,0.196970
0,SL1 ONLY,202012,AMERICAS,<NA>,<NA>,<NA>,90,632,0.142405
0,SL1_CT,202012,AMERICAS,<NA>,<NA>,FIELD,65,522,0.124521
...,...,...,...,...,...,...,...,...,...
119,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_MEA,COUNTRY_KSA,FIELD,1,2,0.500000
120,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_MEA,COUNTRY_KSA,TARGETING,3,8,0.375000
121,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_SP,SP_GERMANY_CENTRAL,TARGETING,1,3,0.333333
122,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_SP,SP_MEA,FIELD,1,3,0.333333


In [41]:
list(load_table['Report Month'].unique())

[202012,
 202101,
 202102,
 202103,
 202104,
 202105,
 202106,
 202107,
 202108,
 202109,
 202110,
 202111,
 202112,
 202201]

In [42]:
load_table.columns

Index(['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name',
       'Conversion Ratio %'],
      dtype='object')

In [39]:
load_table.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\Conversion Rates.xlsx', index=False)
load_table.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\Conversion Rate\Conversion Rates_' + str(use_list[-2])  + '.xlsx', index=False)
load_table.to_excel(r'C:\Users\phsheari\Documents\Compass Conversion Ratios\Conversion Rates_' + str(use_list[-2])  + '.xlsx', index=False)

In [44]:
load_table['Dimension Grouping'].unique()

array(['OVERALL', 'CT ONLY', 'SL1 ONLY', 'SL1_CT', 'SL1_SL2',
       'SL1_SL2_CT', 'SL1_SL2_SL3', 'SL1_SL2_SL3_CT'], dtype=object)

In [45]:
load_table

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Sales Level 3,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,OVERALL,202012,<NA>,<NA>,<NA>,<NA>,91,670,0.135821
0,CT ONLY,202012,<NA>,<NA>,<NA>,FIELD,66,557,0.118492
1,CT ONLY,202012,<NA>,<NA>,<NA>,TARGETING,26,132,0.196970
0,SL1 ONLY,202012,AMERICAS,<NA>,<NA>,<NA>,91,632,0.143987
0,SL1_CT,202012,AMERICAS,<NA>,<NA>,FIELD,66,522,0.126437
...,...,...,...,...,...,...,...,...,...
120,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_MEA,COUNTRY_KSA,FIELD,1,2,0.500000
121,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_MEA,COUNTRY_KSA,TARGETING,3,8,0.375000
122,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_SP,SP_GERMANY_CENTRAL,TARGETING,1,3,0.333333
123,SL1_SL2_SL3_CT,202201,EMEAR-REGION,EMEAR_SP,SP_MEA,FIELD,1,3,0.333333


In [49]:
load_table.loc[(load_table['Sales Level 2'].str.contains('GES', na=False, case=False))]

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Sales Level 3,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %


In [41]:
usps2 = load_table.loc[(load_table['Dimension Grouping']=='SL1_SL2_CT') & (load_table['Sales Level 2']=='US PS MARKET SEGMENT')  &  (load_table['Report Month']==202201)]
usps3 = load_table.loc[(load_table['Dimension Grouping']=='SL1_SL2_SL3_CT') & (load_table['Sales Level 2']=='US PS MARKET SEGMENT')  &  (load_table['Report Month']==202201)]

In [42]:
with pd.ExcelWriter(r'C:\Users\phsheari\Documents\Compass Conversion Ratios\Conversion Ratio USPS_202201.xlsx') as writer:  
    usps2.to_excel(writer, sheet_name = 'SalesLevel2', index=False)
    usps3.to_excel(writer, sheet_name = 'SalesLevel3', index = False)

## Data Audit Area

In [40]:
for fiscal_period_id in use_list[:-1]:
    max_tracker_readout_date = pd.to_datetime(caldf.loc[caldf['finbi_fiscal_period_id'] == fiscal_period_id]['date'].max()) # The absolute MAX date for the finbi period
    min_tracker_readout_date = max_tracker_readout_date + dt.timedelta(days = -333)
    revised_max_tracker_readout_date = max_tracker_readout_date + dt.timedelta(days = -30)   #reset the MAX date for the period to be 30 days less, to account for the bookings minimum of 30 days out, for that booking window
    
    oldest_fiscal_week_id = caldf.loc[caldf['date']==min_tracker_readout_date.isoformat()[:10]]['finbi_fiscal_week_id'].values[0]
    thirty_days_ago_week_id = caldf.loc[caldf['date']==revised_max_tracker_readout_date.isoformat()[:10]]['finbi_fiscal_week_id'].values[0]
    print(f'{fiscal_period_id}: {min_tracker_readout_date} as weekID {oldest_fiscal_week_id}, {max_tracker_readout_date}, {revised_max_tracker_readout_date} as weekID {thirty_days_ago_week_id}')

202012: 2019-08-27 00:00:00 as weekID 2020021, 2020-07-25 00:00:00, 2020-06-25 00:00:00 as weekID 2020121
202101: 2019-09-24 00:00:00 as weekID 2020031, 2020-08-22 00:00:00, 2020-07-23 00:00:00 as weekID 2020125
202102: 2019-10-22 00:00:00 as weekID 2020035, 2020-09-19 00:00:00, 2020-08-20 00:00:00 as weekID 2021014
202103: 2019-11-26 00:00:00 as weekID 2020051, 2020-10-24 00:00:00, 2020-09-24 00:00:00 as weekID 2021031
202104: 2019-12-24 00:00:00 as weekID 2020061, 2020-11-21 00:00:00, 2020-10-22 00:00:00 as weekID 2021035
202105: 2020-01-21 00:00:00 as weekID 2020065, 2020-12-19 00:00:00, 2020-11-19 00:00:00 as weekID 2021044
202106: 2020-02-25 00:00:00 as weekID 2020081, 2021-01-23 00:00:00, 2020-12-24 00:00:00 as weekID 2021061
202107: 2020-03-31 00:00:00 as weekID 2020092, 2021-02-27 00:00:00, 2021-01-28 00:00:00 as weekID 2021071
202108: 2020-04-28 00:00:00 as weekID 2020101, 2021-03-27 00:00:00, 2021-02-25 00:00:00 as weekID 2021075
202109: 2020-06-02 00:00:00 as weekID 2020112,

In [41]:
fin_sl1_sl2_ct    = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= 2020021) & (fins['Fiscal Week ID'] <= 2020121)) ].groupby(['Sales Level 1','Sales Level 2','Campaign Type'])[['FINBI Name']].nunique()

In [42]:
trkr_condition0 = (tracker['Deliverability']=='Good')
trkr_condition1A = (tracker['Date Readout Done']>= '2020-01-21')
trkr_condition1B = (tracker['Date Readout Done']<= '2020-11-19')
trkr_condition2 = ~(tracker['Compass Campaign Name'].isin(removed_campaigns))
trkr_condition3 = ~(tracker['Readout Delivery Status'].isin(['Declined','Postponed']))
trkr_condition4 = (tracker['Lvl2 (Region)']=='GES')
trkr_condition5 = (~(tracker['Compass Advisor'].isin(['N/A','Expired','None'])) | (tracker['Compass Advisor'] != None))

In [43]:
tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4 & trkr_condition5][['Customer Name','Lvl1','Lvl2 (Region)','Campaign Type','Compass Advisor','Readout Delivery Status','Items Recommended','Readout FY','Readout FM','Date Readout Done',]].groupby(['Campaign Type','Customer Name']).count()[59:]

Lvl1  Lvl2 (Region)  Compass Advisor  \
Campaign Type Customer Name                                                  
REACTIVE      VERITAS TECHNOLOGY INC     2              2                2   
              WORKDAY INC                1              1                1   

                                      Readout Delivery Status  \
Campaign Type Customer Name                                     
REACTIVE      VERITAS TECHNOLOGY INC                        2   
              WORKDAY INC                                   1   

                                      Items Recommended  Readout FY  \
Campaign Type Customer Name                                           
REACTIVE      VERITAS TECHNOLOGY INC                  2           2   
              WORKDAY INC                             1           1   

                                      Readout FM  Date Readout Done  
Campaign Type Customer Name                                          
REACTIVE      VERITAS TECHNOLOGY INC           2                  2  
              WORKDAY INC                      1                  1

In [44]:
compass_sl1_sl2_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1','Lvl2 (Region)','Campaign Type'])[['Customer Name']].nunique()

In [45]:
fin_sl1_sl2_ct

FINBI Name
Sales Level 1 Sales Level 2        Campaign Type            
Americas      CANADA               PROACTIVE               1
                                   REACTIVE                2
              GES                  PROACTIVE               6
                                   REACTIVE               17
              LATIN AMERICA        PROACTIVE               2
                                   REACTIVE               10
              US COMMERCIAL        PROACTIVE               8
                                   REACTIVE               10
              US PS Market Segment REACTIVE                3

In [46]:
compass_sl1_sl2_ct

Customer Name
Lvl1         Lvl2 (Region)        Campaign Type               
AMERICAS     AMERICAS_SP          PROACTIVE                  3
                                  REACTIVE                  11
             CANADA               PROACTIVE                  5
                                  REACTIVE                  56
             GES                  PROACTIVE                  6
                                  REACTIVE                  55
             LATIN AMERICA        PROACTIVE                  8
                                  REACTIVE                 110
             US COMMERCIAL        PROACTIVE                261
                                  REACTIVE                 337
             US PS MARKET SEGMENT PROACTIVE                 28
                                  REACTIVE                  63
APJC         ANZ AREA             PROACTIVE                 29
                                  REACTIVE                  26
             APJ_SP               PROACTIVE                  3
             ASEAN_AREA           PROACTIVE                  5
                                  REACTIVE                   5
             GREATER_CHINA        PROACTIVE                 19
                                  REACTIVE                   4
             INDIA_AREA           PROACTIVE                  3
                                  REACTIVE                   6
             JAPAN__              PROACTIVE                 10
             ROK_AREA             PROACTIVE                  2
EMEAR-REGION EMEAR-CENTRAL        PROACTIVE                  1
                                  REACTIVE                  10
             EMEAR-NORTH          PROACTIVE                 15
                                  REACTIVE                  10
             EMEAR-SOUTH          PROACTIVE                 26
                                  REACTIVE                  15
             EMEAR-UKI            PROACTIVE                 49
                                  REACTIVE                  64
             EMEAR_GERMANY        PROACTIVE                  1
                                  REACTIVE                   6
             EMEAR_MEA            PROACTIVE                  1
                                  REACTIVE                  15

In [47]:
# caldf.loc[caldf['date']==revised_max_tracker_readout_date.isoformat()[:10]]['finbi_fiscal_week_id'].values[0]

In [48]:
# caldf.head(2)

In [49]:
# fins['Report Month'] = fiscal_period_id

In [50]:
# fins.columns

In [51]:
#((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id))

In [52]:
# fin_sl1           = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Sales Level 1'])[['FINBI Name']].nunique()
# fin_ct            = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Campaign Type'])[['FINBI Name']].nunique()
# fin_sl1_ct        = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Sales Level 1','Campaign Type'])[['FINBI Name']].nunique()
# fin_sl1_sl2_ct    = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Sales Level 1','Sales Level 2','Campaign Type'])[['FINBI Name']].nunique()
# fin_sl1_sl2       = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Sales Level 1','Sales Level 2'])[['FINBI Name']].nunique()
# fin_sl1_sl2_sl3   = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Sales Level 1','Sales Level 2','Sales Level 3'])[['FINBI Name']].nunique()
# fin_sl1_sl2_sl3_ct = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ].groupby(['Sales Level 1','Sales Level 2','Sales Level 3','Campaign Type'])[['FINBI Name']].nunique()
# overall_numerator = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) ][['FINBI Name']].nunique()[0]

In [53]:
# fin_sl1_ct.reset_index(inplace=True)
# fin_ct.reset_index(inplace=True)
# fin_sl1.reset_index(inplace=True)
# fin_sl1_sl2_ct.reset_index(inplace=True)
# fin_sl1_sl2.reset_index(inplace=True)
# fin_sl1_sl2_sl3.reset_index(inplace=True)
# fin_sl1_sl2_sl3_ct.reset_index(inplace=True)

In [54]:
# overall_numerator

### Set the SL and Campaign Dimension Level on the various dataframes

In [55]:
# fin_sl1_ct['Report Month'] = fiscal_period_id
# fin_sl1_ct['Dimension Grouping'] = 'SL1_CT'

In [56]:
# fin_ct['Report Month'] = fiscal_period_id
# fin_ct['Dimension Grouping'] = 'CT ONLY'

In [57]:
# fin_sl1['Report Month'] = fiscal_period_id
# fin_sl1['Dimension Grouping'] = 'SL1 ONLY'

In [58]:
# fin_sl1_sl2_ct['Report Month'] = fiscal_period_id
# fin_sl1_sl2_ct['Dimension Grouping'] = 'SL1_SL2_CT'

In [59]:
# fin_sl1_sl2['Report Month'] = fiscal_period_id
# fin_sl1_sl2['Dimension Grouping'] = 'SL1_SL2'

In [60]:
# fin_sl1_sl2_sl3['Report Month'] = fiscal_period_id
# fin_sl1_sl2_sl3['Dimension Grouping'] = 'SL1_SL2_SL3'

In [61]:
# fin_sl1_sl2_sl3_ct['Report Month'] = fiscal_period_id
# fin_sl1_sl2_sl3_ct['Dimension Grouping'] = 'SL1_SL2_SL3_CT'

In [62]:
# fin_sl1_sl2_sl3_ct

In [63]:
# fin_sl1['Sales Level 1']        = fin_sl1['Sales Level 1'].str.upper()
# fin_ct['Campaign Type']         = fin_ct['Campaign Type'].str.upper()
# fin_sl1_ct['Sales Level 1']     = fin_sl1_ct['Sales Level 1'].str.upper()
# fin_sl1_sl2_ct['Sales Level 1'] = fin_sl1_sl2_ct['Sales Level 1'].str.upper()
# fin_sl1_sl2_ct['Sales Level 2'] = fin_sl1_sl2_ct['Sales Level 2'].str.upper()
# fin_sl1_sl2['Sales Level 1']    = fin_sl1_sl2['Sales Level 1'].str.upper()
# fin_sl1_sl2['Sales Level 2']    = fin_sl1_sl2['Sales Level 2'].str.upper()
# fin_sl1_sl2_sl3['Sales Level 1']    = fin_sl1_sl2_sl3['Sales Level 1'].str.upper()
# fin_sl1_sl2_sl3['Sales Level 2']    = fin_sl1_sl2_sl3['Sales Level 2'].str.upper()
# fin_sl1_sl2_sl3['Sales Level 3']    = fin_sl1_sl2_sl3['Sales Level 3'].str.upper()
# fin_sl1_sl2_sl3_ct['Sales Level 1']    = fin_sl1_sl2_sl3_ct['Sales Level 1'].str.upper()
# fin_sl1_sl2_sl3_ct['Sales Level 2']    = fin_sl1_sl2_sl3_ct['Sales Level 2'].str.upper()
# fin_sl1_sl2_sl3_ct['Sales Level 3']    = fin_sl1_sl2_sl3_ct['Sales Level 3'].str.upper()

In [64]:
# removed_campaigns = ['REACTIVE - ACC DESIGNATION','REACTIVE - PARTNER','REACTIVE - GROWTH INITIATIVE',
#                      'PROACTIVE - TAC LEAD','PROACTIVE - CS RESEARCH','PROACTIVE - PRESALES ALIGN PITSTOP EFFORT']

In [65]:
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('UKI', case=False)), 'Lvl2 (Region)'] = 'EMEAR-UKI'

In [66]:
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('central', case=False)), 'Lvl2 (Region)'] = 'EMEAR-CENTRAL'

In [67]:
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('north', case=False)), 'Lvl2 (Region)'] = 'EMEAR-NORTH'

In [68]:
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('south', case=False)), 'Lvl2 (Region)'] = 'EMEAR-SOUTH'

In [69]:
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('ANZ', case=False)), 'Lvl2 (Region)'] = 'ANZ AREA'

In [70]:
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('japan', case=False)), 'Lvl2 (Region)'] = 'JAPAN__'

In [71]:
# tracker.columns

In [72]:
# trkr_condition1A = (tracker['Date Readout Done']>= min_tracker_readout_date)
# trkr_condition1B = (tracker['Date Readout Done']<= revised_max_tracker_readout_date)
# trkr_condition2 = ~(tracker['Compass Campaign Name'].isin(removed_campaigns))
# trkr_condition3 = ~(tracker['Readout Delivery Status'].isin(['Declined','Postponed']))
# #trkr_condition4 = ~(tracker['Lvl2 (Region)']=='LATAM')
# trkr_condition5 = (tracker['is_Targeting_Campaign']==True)

In [73]:
# compass_sl1         = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1'])[['Customer Name']].nunique()
# compass_ct          = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Campaign Type'])[['Customer Name']].nunique()
# compass_sl1_ct      = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1','Campaign Type'])[['Customer Name']].nunique()
# compass_sl1_sl2_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1','Lvl2 (Region)','Campaign Type'])[['Customer Name']].nunique()
# compass_sl1_sl2     = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1','Lvl2 (Region)'])[['Customer Name']].nunique()
# compass_sl1_sl2_sl3     = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1','Lvl2 (Region)','Lvl3'])[['Customer Name']].nunique()
# compass_sl1_sl2_sl3_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ].groupby(['Lvl1','Lvl2 (Region)','Lvl3','Campaign Type'])[['Customer Name']].nunique()
# overall_denominator = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 ][['Customer Name']].nunique()[0]

In [74]:
# ### Sanity Check
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('UKI', case=False)), 'Lvl2 (Region)'] = 'EMEAR-UKI'

In [75]:
# compass_sl1.reset_index(inplace=True)
# compass_ct.reset_index(inplace=True)
# compass_sl1_ct.reset_index(inplace=True)
# compass_sl1_sl2_ct.reset_index(inplace=True)
# compass_sl1_sl2.reset_index(inplace=True)
# compass_sl1_sl2_sl3.reset_index(inplace=True)
# compass_sl1_sl2_sl3_ct.reset_index(inplace=True)

In [76]:
# compass_sl1.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
# compass_ct.rename(columns = {'Customer Name':'Compass Name'}, inplace=True)
# compass_sl1_ct.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
# compass_sl1_sl2_ct.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Customer Name':'Compass Name'}, inplace=True)
# compass_sl1_sl2.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Customer Name':'Compass Name'}, inplace=True)
# compass_sl1_sl2_sl3.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2', 'Lvl3':'Sales Level 3','Customer Name':'Compass Name'}, inplace=True)
# compass_sl1_sl2_sl3_ct.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Lvl3':'Sales Level 3','Customer Name':'Compass Name'}, inplace=True)

In [77]:
# fin_sl1_sl2_sl3_ct

In [78]:
# combined_sl1 = pd.merge(fin_sl1, compass_sl1, on = ['Sales Level 1'])

In [79]:
# combined_ct = pd.merge(fin_ct, compass_ct, on=['Campaign Type'])

In [80]:
# combined_sl1_ct = pd.merge(fin_sl1_ct, compass_sl1_ct, on = ['Sales Level 1','Campaign Type'])

In [81]:
# combined_sl1_sl2_ct = pd.merge(fin_sl1_sl2_ct,compass_sl1_sl2_ct, on = ['Sales Level 1', 'Sales Level 2','Campaign Type'])

In [82]:
# combined_sl1_sl2 = pd.merge(fin_sl1_sl2,compass_sl1_sl2, on = ['Sales Level 1', 'Sales Level 2'])

In [83]:
# combined_sl1_sl2_sl3 = pd.merge(fin_sl1_sl2_sl3,compass_sl1_sl2_sl3, on = ['Sales Level 1', 'Sales Level 2','Sales Level 3'])

In [84]:
# combined_sl1_sl2_sl3_ct = pd.merge(fin_sl1_sl2_sl3_ct,compass_sl1_sl2_sl3_ct, on = ['Sales Level 1', 'Sales Level 2','Sales Level 3','Campaign Type'])

In [85]:
# combined_sl1_sl2_sl3_ct

In [86]:
# combined_sl1_sl2_ct['Conversion Ratio %'] = (combined_sl1_sl2_ct['FINBI Name'] / combined_sl1_sl2_ct['Compass Name']) #* 100
# combined_sl1_ct['Conversion Ratio %'] = (combined_sl1_ct['FINBI Name'] / combined_sl1_ct['Compass Name']) #* 100
# combined_sl1['Conversion Ratio %'] = (combined_sl1['FINBI Name'] / combined_sl1['Compass Name']) #* 100
# combined_ct['Conversion Ratio %'] = (combined_ct['FINBI Name'] / combined_ct['Compass Name']) #* 100
# combined_sl1_sl2['Conversion Ratio %'] = (combined_sl1_sl2['FINBI Name'] / combined_sl1_sl2['Compass Name']) #* 100
# combined_sl1_sl2_sl3['Conversion Ratio %'] = (combined_sl1_sl2_sl3['FINBI Name'] / combined_sl1_sl2_sl3['Compass Name']) #* 100
# combined_sl1_sl2_sl3_ct['Conversion Ratio %'] = (combined_sl1_sl2_sl3_ct['FINBI Name'] / combined_sl1_sl2_sl3_ct['Compass Name']) #* 100

In [87]:
# combined_ct['Sales Level 1'] = pd.NA
# combined_ct['Sales Level 2'] = pd.NA
# combined_ct['Sales Level 3'] = pd.NA

In [88]:
# combined_ct = combined_ct[['Dimension Grouping', 'Report Month','Sales Level 1','Sales Level 2','Sales Level 3','Campaign Type', 'FINBI Name',  'Compass Name','Conversion Ratio %']]

In [89]:
# combined_sl1_ct['Sales Level 2'] = pd.NA
# combined_sl1_ct['Sales Level 3'] = pd.NA

In [90]:
# combined_sl1_ct = combined_sl1_ct[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [91]:
# combined_sl1['Sales Level 2'] = pd.NA
# combined_sl1['Campaign Type'] = pd.NA
# combined_sl1['Sales Level 3'] = pd.NA

In [92]:
## combined_sl1 = combined_sl1[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [93]:
# combined_sl1_sl2_ct['Sales Level 3'] = pd.NA

In [94]:
# combined_sl1_sl2_ct = combined_sl1_sl2_ct[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [95]:
# combined_sl1_sl2['Campaign Type'] = pd.NA
# combined_sl1_sl2['Sales Level 3'] = pd.NA

In [96]:
# combined_sl1_sl2 = combined_sl1_sl2[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [97]:
# combined_sl1_sl2_sl3['Campaign Type'] = pd.NA

In [98]:
# combined_sl1_sl2_sl3 = combined_sl1_sl2_sl3[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [99]:
# combined_ct

In [100]:
# combined_sl1

In [101]:
# combined_sl1_ct

In [102]:
# combined_sl1_sl2

In [103]:
# combined_sl1_sl2_ct

In [104]:
# combined_sl1_sl2_sl3

In [105]:
# combined_sl1_sl2_sl3_ct

### Campaign Type Only

In [106]:
# combined_ct.columns

### Sales Level 1 Only

In [107]:
# combined_sl1

### SL1 = Sales Level 1 and CT = Campaign Type

In [108]:
# combined_sl1_ct

In [109]:
# with pd.ExcelWriter(r'C:\Users\phsheari\Documents\FIN BI Data\Conversion Rate\ConversionRates_' + file_run_date + '_.xlsx') as writer:
#     combined_sl1_ct.to_excel(writer, sheet_name = 'Theater_CampaignType', index=False)
#     combined_sl1_sl2_ct.to_excel(writer, sheet_name = 'Theater_SL2_CampaignType', index=False)
#     combined_sl1.to_excel(writer, sheet_name = 'Theater_Only', index=False)
#     combined_ct.to_excel(writer, sheet_name = 'CampaignType_Only', index=False)      

### SL1 = Sales Level 1 and SL2 = Sales Level 2 and CT = Campaign Type

In [110]:
# combined_sl1_sl2_ct

In [111]:
# combined_sl1_sl2_ct.to_excel(r'C:\Users\phsheari\Documents\Francisco\USCOMMERCIAL_ConversionRates.xlsx', index=False)

In [112]:
# overall_conversion_ratio = (overall_numerator / overall_denominator) #* 100
# print(overall_numerator, overall_denominator)

In [113]:
# overall_list = [['OVERALL',fiscal_period_id, pd.NA, pd.NA, pd.NA, pd.NA, overall_numerator, overall_denominator, overall_conversion_ratio]]
# overall_columns = ['Dimension Grouping', 'Report Month','Sales Level 1','Sales Level 2','Sales Level 3','Campaign Type','FINBI Name','Compass Name','Conversion Ratio %']
# overall = pd.DataFrame(data = overall_list, columns = overall_columns)

In [114]:
# overall

In [115]:
# overall_conversion_ratio

In [116]:
# temp_table = pd.concat([overall,combined_ct,combined_sl1, combined_sl1_ct, combined_sl1_sl2, combined_sl1_sl2_ct, combined_sl1_sl2_sl3, combined_sl1_sl2_sl3_ct])

In [117]:
# load_table.append(temp_table)

In [118]:
# load_table['Report Month'] = '202112'

In [119]:
# load_table

In [120]:
# load_table.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\Conversion Rate\ConversionRates_' + fiscal_month_id + '+_SL3_LATAM.xlsx', sheet_name=fiscal_month_id, index=False)

In [121]:
# load_table['Sales Level 2'].unique()

In [122]:
# load_table.loc[load_table['Sales Level 2'] == 'US PS MARKET SEGMENT']

In [123]:
# len(sorted(list(fins['Fiscal Week ID'].unique())))

In [124]:
# sorted(list(fins['Fiscal Week ID'].unique()))[0:37]

In [125]:
# 2021012 - 2020013